In [7]:
from os import listdir

def fileids(path):
    fileList = list()
    
    path = path + ('' if path[-1] == '/' else '/')
    
    for f in listdir(path):
        if f.endswith('.txt'):
            fileList.append(path+f)
            
    return fileList

In [8]:
import re

p1 = re.compile('[a-zA-Z0-9_-]+@(?:[.]?[a-zA-Z0-9_-]+)+')
p2 = re.compile('[{}]'.format(re.escape('''"'=ㆍⓒ'"''')))
p3 = re.compile('[\(\[].+?[\)\]]')
p4 = re.compile('[a-zA-Z]+')
p5 = re.compile('\s+')
# p5.sub(' ', p4.sub(' ', p3.sub(' ', p2.sub(' ', p1.sub(' ', d))))).strip()

In [11]:
corpus = list()

for f in fileids(r"C:\Users\dlel2\Documents\GitHub\INISW_2023\텍스트데이터처리\news"):
    with open(f, 'r', encoding='utf8') as fp:
        corpus.append(p5.sub(' ', 
            p4.sub(' ', 
            p3.sub(' ', 
            p2.sub(' ', 
            p1.sub(' ', fp.read()))))).strip())

In [12]:
def ngram2(t, n=2):
    tokens = list(t)
    gram = list()
 
    if len(tokens) < n:
        return tokens
    for i in range(len(tokens)-(n-1)):
        gram.append(''.join(tokens[i:i+n]))
    return gram

In [13]:
from collections import Counter

gram1 = Counter(ngram2('\n'.join(corpus),1))
gram2 = Counter(ngram2('\n'.join(corpus),2))
gram3 = Counter(ngram2('\n'.join(corpus),3))
gram4 = Counter(ngram2('\n'.join(corpus),4))

gram_finder = {1:gram1, 2:gram2, 3:gram3, 4:gram4}

def auto_space(s, base=2, verbose=True):
    result = ''
    
    if base not in [2,3,4]:
        return s
    
    if len(s) < base:
        return s
    
    freq1 = gram_finder[base-1]
    freq2 = gram_finder[base]
    
    i = base-1
    k = s[:i]
    while i < len(s):
        c = dict(filter(lambda r:re.match(k, r[0]), freq2.items()))
        
        result += k[0]

        if k not in freq1:
            if verbose:
                print('Not found:', k, 'in Freq1')
            i += 1
            k = k[1:] + s[i-1]
            continue
            
        if len(c) == 0:
            if verbose:
                print('Not found:', k, 'in Freq2')
            i += 1
            k = k[1:] + s[i-1]
            continue
        
        t = max(c, key=c.get)
        f = c[t]
        
        if verbose:
            print('i:{} | K:{} | P:{:.2f} - [{}]'.format(i, k, f/freq1[k], t))
        
        if t[-1] == ' ':
            k = t[1:]
        else:
            i += 1
            k = k[1:] + s[i-1]
            
    result += s[i-base+1:]
    
    return result

s = '가족관계등록법은 가정폭력 피해자가 가해자로 지정한 배우자 직계혈족이 피해자 본인의 가족관계증명서 등을 받아 갈 수 없도록 못 박았다.'
s = re.sub(' ', '', s)
auto_space(s, 4, False)

'가족관계등록 법은 가정폭력 피해 자가가해자로지정한 배우자 직계혈족이피해자본인의 가족 관계증명서등을받아갈수없도록 못박았다.'